In [10]:
%pip install numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install --force-reinstall matplotlib

  Using cached matplotlib-3.9.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.0-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.54.1-cp311-cp311-win_amd64.whl.metadata (167 kB)
  Using cached kiwisolver-1.4.7-cp311-cp311-win_amd64.whl.metadata (6.4 kB)
  Using cached numpy-2.1.2-cp311-cp311-win_amd64.whl.metadata (59 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached pillow-11.0.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
Using cached matplotlib-3.9.2-cp311-cp311-win_amd64.whl (7.8 MB)
Using cached contourpy-1.3.0-cp311-cp311-win_amd64.whl (217 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.54.1-c

error: uninstall-no-record-file

× Cannot uninstall matplotlib 3.9.2
╰─> The package's contents are unknown: no RECORD file was found for matplotlib.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps matplotlib==3.9.2


In [11]:
import numpy as np
from scipy.optimize import minimize
from scipy import linalg

import matplotlib.pyplot as plt
import tikzplotlib

# np.random.seed(0)
N = 100
half_n = N // 2

r = 10
x0_gt, y0_gt = 2, 3  # Center
s = r / 16
t = np.random.uniform(0, 2 * np.pi, half_n)
n = s * np.random.randn(half_n)
x, y = x0_gt + (r + n) * np.cos(t), y0_gt + (r + n) * np.sin(t)
X_circ = np.hstack((x.reshape(half_n, 1), y.reshape(half_n, 1)))

s = 1.
m, b = 1, 2
x = np.linspace(-12, 12, half_n)
y = m * x + b + s * np.random.randn(half_n)
X_line = np.hstack((x.reshape(half_n, 1), y.reshape(half_n, 1)))

X = np.vstack((X_circ, X_line))  # All points

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.scatter(X_line[:, 0], X_line[:, 1], label='Line')
ax.scatter(X_circ[:, 0], X_circ[:, 1], label='Circle')

circle_gt = plt.Circle((x0_gt, y0_gt), r, color='g', fill=False, label='Ground truth circle')
ax.add_patch(circle_gt)
ax.plot((x0_gt), (y0_gt), '+', color='g')

x_min, x_max = ax.get_xlim()
x_ = np.array([x_min, x_max])
y_ = m * x_ + b
plt.plot(x_, y_, color='m', label='Ground truth line')

plt.legend()
plt.show()


ImportError: DLL load failed while importing _c_internal_utils: The specified module could not be found.

In [1]:
# Helper function to compute distance of points from a line
def compute_line_distance(points, a, b, d):
    return np.abs(a * points[:, 0] + b * points[:, 1] + d) / np.sqrt(a**2 + b**2)

# RANSAC algorithm to estimate a line
def ransac_line(X, threshold, num_iterations):
    best_inliers = []
    best_params = None

    for i in range(num_iterations):
        # Randomly select two points to define the line
        sample = X[np.random.choice(X.shape[0], 2, replace=False), :]

        # Compute line parameters: ax + by + d = 0
        # [a,b] is normal to the line
        p1, p2 = sample[0], sample[1]
        a = p2[1] - p1[1]
        b = p1[0] - p2[0]
        d = -(a * p1[0] + b * p1[1])
        norm = np.sqrt(a**2 + b**2)
        a, b, d = a/norm, b/norm, d/norm  # Normalize to satisfy ∥[a,b]⊺∥=1

        # Calculate the distances of all points to the line
        distances = compute_line_distance(X, a, b, d)

        # Identify inliers
        inliers = X[distances < threshold]

        # Keep the model with the most inliers
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_params = (a, b, d)

    return best_params, best_inliers

# Use the provided noisy point set
# Set parameters for RANSAC
ransac_iterations = 100
ransac_threshold = 0.5  # Adjust based on data

# Estimate the line using RANSAC
best_line_params, best_line_inliers = ransac_line(X, ransac_threshold, ransac_iterations)

# Plot the points and the estimated line
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.scatter(X[:, 0], X[:, 1], label='Data points')
a, b, d = best_line_params
x_vals = np.linspace(X[:, 0].min(), X[:, 0].max(), 100)
y_vals = -(a * x_vals + d) / b
ax.plot(x_vals, y_vals, color='r', label='Estimated line (RANSAC)')
ax.scatter(best_line_inliers[:, 0], best_line_inliers[:, 1], color='g', label='Inliers')
plt.legend()
plt.show()


NameError: name 'X' is not defined